In [5]:

import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib
import matplotlib.pyplot as plt

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 750)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.227574219e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 10000


In [10]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada_l1 = read_dada("../../../data/J1644-4559_2024-02-16T11:21:22.092_54_73532270706688.dada")
print(dada_l1.get_header())
dada_l1 = dada_l1.combined_data()
dada_l1 = dada_l1[0:100000, core_indices, 47:54, 0:2]
X_l1 = dada_l1



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 3392, 'Lowest_freq (MHz)': 1564.875}


In [11]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("../../../data/gps.dada")
print(dada.get_header())
dada = dada.combined_data()
dada = dada[0:100000, core_indices, 47:54, 0:2]
X = dada


{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [12]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np
import astropy.units as u

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

lon_str = "21d26m38s"   # degrees, minutes, seconds format
lat_str = "-30d42m39.8s"  # negative for south

# Altitude (optional, in meters)
height = 1086.6 * u.m

# Create location
location = EarthLocation.from_geodetic(lon=lon_str, lat=lat_str, height=height)

print(location)

(5109360.133321233, 2006852.5860429057, -3238948.127478875) m


/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_52332/3372582233.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp)


In [13]:


# Observer location (MeerKAT)
obs_location = EarthLocation(lat=lat*u.deg,
                         lon=lon*u.deg,
                         height=elevation_m*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print("Source RA/DEC")
print(f"Altitude: {altaz.alt:.2f}")
print(f"Azimuth: {altaz.az:.2f}")

theta_source = 90 - altaz.alt.degree
phi_source = (450 - altaz.az.degree) % 360

print(theta_source)
print(phi_source)

Source RA/DEC
Altitude: 24.03 deg
Azimuth: 229.33 deg
65.97485141274609
220.6699421578765


In [14]:
X.shape

(100000, 43, 7, 2)

In [15]:
from skyfield.api import wgs84, Topos
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json

ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')
meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site = meerkat
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    # any of them should give essentially the same result.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt.degrees
    event_dict['az'] = az.degrees
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
print(len(sats_visible))
df.with_columns(theta=pl.lit(90) - pl.col('alt'), phi=(pl.lit(450) - pl.col('az'))%360).sort(by="alt", descending=False)


Loaded 54 satellites
10


name,alt,az,theta,phi
str,f64,f64,f64,f64
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,85.903507,308.897567
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,81.951034,44.98524
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,73.357624,358.366296
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,67.443908,204.545606
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,53.142185,69.596966
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,52.019064,140.39203
"""NAVSTAR 64 (USA 206)""",44.369909,123.403165,45.630091,326.596835
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,44.627335,230.619031
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,40.096014,309.934402


In [16]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


In [17]:
X.shape

(100000, 43, 7, 2)

In [23]:
num_timesteps = 10_000

X2 = X[0:num_timesteps, :, :, :]
f = np.array([1.226947266e9, 1.22715625e9, 1.227365234e9, 1.227574219e9, 1.227783203e9, 1.227992188e9, 1.228201172e9])

wv = constants.c / f

X_adj = X2.copy()

for channel in range(X.shape[2]):
    array_factor = array.steering_vector([np.deg2rad(phi_source), np.deg2rad(theta_source)], wv[channel])
    for i, shift in enumerate(array_factor):

        X_adj[:, i, channel, :] = X_adj[:, i, channel, :] * array_factor[i].conj()


X2_l1 = X_l1[0:num_timesteps, :, :, :]
f2 = np.array([1.574697266e9, 1.57490625e9, 1.575115234e9, 1.575324219e9, 1.575533203e9, 1.575742188e9, 1.575951172e9])
wv2 = constants.c / f2

X_l1_adj = X2_l1.copy()

for channel in range(X_l1_adj.shape[2]):
    array_factor = array.steering_vector([np.deg2rad(phi_source), np.deg2rad(theta_source)], wv2[channel])
    for i, shift in enumerate(array_factor):

        X_l1_adj[:, i, channel, :] = X_l1_adj[:, i, channel, :] * array_factor[i].conj()



In [24]:
X_l1_adj.shape

(10000, 43, 7, 2)

In [ ]:
from tqdm import tqdm
from itertools import product
from collections import defaultdict

num_antennas = len(core_indices)
theta_steps = 750#6_000
phi_steps = 750#24_000

theta_range = np.linspace(0, 86, 87*6)
phi_range = np.linspace(180, 270, 160*6)

output = []

acms = defaultdict(list)
acms_l1 = defaultdict(list)

num_interferers = 7

for channel in range(X_adj.shape[2]):
    for pol in range(0,2):
        acm = X_adj[:, :, channel, pol].conj().T @ X_adj[:, :, channel, pol] / (num_timesteps - 1)
    
        evals, evecs = np.linalg.eigh(acm)
    
        assert len(evals) == num_antennas
        
    
        noise_matrix = evecs[:, :-num_interferers] @ evecs[:, :-num_interferers].conj().T
        acms[channel].append(noise_matrix)

for channel in range(X_l1_adj.shape[2]):
    for pol in range(0,2):
        acm = X_l1_adj[:, :, channel, pol].conj().T @ X_l1_adj[:, :, channel, pol] / (num_timesteps - 1)
    
        evals, evecs = np.linalg.eigh(acm)
    
        assert len(evals) == num_antennas
    
        noise_matrix = evecs[:, :-num_interferers] @ evecs[:, :-num_interferers].conj().T
        acms_l1[channel].append(noise_matrix)


THRESHOLD_SET = False
threshold = None
iteration = 0

for theta,phi in tqdm(product(theta_range, phi_range), total = len(theta_range) * len(phi_range)):
    Q_sum = 1
    for channel in range(X_adj.shape[2]):
        steer_vec = array.steering_vector([np.deg2rad(phi), np.deg2rad(theta)], wv[channel]).T
        steer_vec_l1 = array.steering_vector([np.deg2rad(phi), np.deg2rad(theta)], wv2[channel]).T
        for pol in range(0,2):
            Q_sum *= 1/np.abs(steer_vec.conj().T @ acms[channel][pol]  @ steer_vec)
            Q_sum *= 1/np.abs(steer_vec_l1.conj().T @ acms_l1[channel][pol]  @ steer_vec_l1)
    Q = Q_sum

    if not THRESHOLD_SET or Q > threshold:
        output.append({"theta": theta, "phi": phi, "Q": Q})

    if not THRESHOLD_SET:
        iteration += 1
        if iteration > 20_000:
            THRESHOLD_SET = True
            threshold = pl.from_records(output)
            threshold = threshold.quantile(0.6)['Q'][0]
    
output = pl.from_records(output)
column_name = "Q"
output = output.with_columns(
            (
                10 * pl.col(column_name).abs().log10()
                - 10 * pl.col(column_name).abs().max().log10()
            ).alias(column_name + "_db")
        )

  9%|█████████                                                                                           | 45329/501120 [01:23<13:42, 553.84it/s]

In [ ]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q_db'].quantile(0.99975)
high_Q = output.filter(pl.col('Q_db') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q_db',      # MUSIC power
    color='Q_db',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

In [ ]:
name	alt	az	theta	phi
str	f64	f64	f64	f64
"NAVSTAR 69 (USA 248)"	4.096493	141.102433	85.903507	308.897567
"NAVSTAR 81 (USA 319)"	8.048966	45.01476	81.951034	44.98524
"NAVSTAR 47 (USA 150)"	16.642376	91.633704	73.357624	358.366296
"NAVSTAR 79 (USA 304)"	22.556092	245.454394	67.443908	204.545606
"NAVSTAR 67 (USA 239)"	36.857815	20.403034	53.142185	69.596966
"NAVSTAR 61 (USA 199)"	37.980936	309.60797	52.019064	140.39203
"NAVSTAR 64 (USA 206)"	44.369909	123.403165	45.630091	326.596835
"NAVSTAR 78 (USA 293)"	45.372665	219.380969	44.627335	230.619031
"NAVSTAR 43 (USA 132)"	49.903986	140.065598	40.096014	309.934402
"NAVSTAR 60 (USA 196)"	76.956355	197.642354	13.043645	252.357646


In [ ]:
output['Q'].min()

In [ ]:
output['Q'].max()

In [53]:
from loguru import logger
logger.info(datetime.now())
theta_range = np.linspace(0, 86, 87*6)
phi_range = np.linspace(180, 270, 160*6)


def compute_Q_joblib(theta_phi):
    theta, phi = theta_phi
    Q_sum = 1
    for channel in range(len(wv)):
        steer_vec = array.steering_vector([np.deg2rad(phi), np.deg2rad(theta)], wv[channel]).T
        steer_vec_l1 = array.steering_vector([np.deg2rad(phi), np.deg2rad(theta)], wv2[channel]).T
        for pol in range(2):
            Q_sum *= 1 / np.abs(steer_vec.conj().T @ acms[channel][pol] @ steer_vec)
            Q_sum *= 1 / np.abs(steer_vec_l1.conj().T @ acms_l1[channel][pol] @ steer_vec_l1)
    return {"theta": theta, "phi": phi, "Q": Q_sum}

from joblib import Parallel, delayed
from tqdm import tqdm

angle_pairs = list(product(theta_range, phi_range))

results = Parallel(n_jobs=-1, backend='loky')(
    delayed(compute_Q_joblib)(pair) for pair in tqdm(angle_pairs)
)

output = pl.from_records(results)
column_name = "Q"
output = output.with_columns(
    (
        10 * pl.col(column_name).abs().log10()
        - 10 * pl.col(column_name).abs().max().log10()
    ).alias(column_name + "_db")
)
logger.info(datetime.now())
logger.info(len(output))


2025-06-11 15:37:18.926 | INFO     | __main__:<module>:2 - 2025-06-11 15:37:18.926569

 90%|███████████████████████████████████████████████████████████████████████████████████████▉          | 449520/501120 [03:39<00:27, 1883.48it/s]Exception ignored in: <function tqdm.__del__ at 0x1203a60c0>
Traceback (most recent call last):
  File "/Users/jsmallwood/Library/Application Support/hatch/env/virtual/spatial-filtering/L8GD9N7U/spatial-filtering/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Users/jsmallwood/Library/Application Support/hatch/env/virtual/spatial-filtering/L8GD9N7U/spatial-filtering/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 501120/501120 [04:06<00:00, 2032.15it/s]
2025-06-11 15:41:26.299 | 

In [54]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q_db'].quantile(0.99975)
high_Q = output.filter(pl.col('Q_db') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q_db',      # MUSIC power
    color='Q_db',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

In [ ]:
name	alt	az	theta	phi
str	f64	f64	f64	f64
"NAVSTAR 69 (USA 248)"	4.096493	141.102433	85.903507	308.897567
"NAVSTAR 81 (USA 319)"	8.048966	45.01476	81.951034	44.98524
"NAVSTAR 47 (USA 150)"	16.642376	91.633704	73.357624	358.366296
"NAVSTAR 79 (USA 304)"	22.556092	245.454394	67.443908	204.545606
"NAVSTAR 67 (USA 239)"	36.857815	20.403034	53.142185	69.596966
"NAVSTAR 61 (USA 199)"	37.980936	309.60797	52.019064	140.39203
"NAVSTAR 64 (USA 206)"	44.369909	123.403165	45.630091	326.596835
"NAVSTAR 78 (USA 293)"	45.372665	219.380969	44.627335	230.619031
"NAVSTAR 43 (USA 132)"	49.903986	140.065598	40.096014	309.934402
"NAVSTAR 60 (USA 196)"	76.956355	197.642354	13.043645	252.357646


In [55]:
output['Q'].min()

2.6941673491130663e-45

In [56]:
output['Q'].max()

1.4986273700648135e-42